In [3]:
import numpy as np
import SimpleITK as sitk
import itk
import os
from scipy import signal

In [4]:
def get_list_of_files(base_dir):

    list_of_lists = []
    patients = os.listdir(base_dir)
    for p in patients:
        if p.startswith("mr_train_") and p.endswith("_image.nii.gz"):
            list_of_lists.append(os.path.join(base_dir, p))
    print("Found %d patients" % len(list_of_lists))
    return list_of_lists

base_dir = '/usr/not-backed-up2/scsad/DL/MedicalDataAugmentationTool/bin/experiments/semantic_segmentation/mmwhs/TODO_mr'
list_of_files = get_list_of_files(base_dir)
dir_img = list_of_files[19]
print(dir_img)

Found 20 patients
/usr/not-backed-up2/scsad/DL/MedicalDataAugmentationTool/bin/experiments/semantic_segmentation/mmwhs/TODO_mr/mr_train_1020_image.nii.gz


In [5]:
def load_img(dir_img):
    # load SimpleITK Image
    img_sitk = sitk.ReadImage(dir_img)
    
    print("Pixel Type    {}".format(img_sitk.GetPixelID()))
    print("Size          {}".format(img_sitk.GetSize()))
    print("Origin        {}".format(img_sitk.GetOrigin()))
    print("Spacing       {}".format(img_sitk.GetSpacing()))
    print("Direction     {}".format(img_sitk.GetDirection()))

    # get pixel arrays from SimpleITK images
    img_npy = sitk.GetArrayFromImage(img_sitk)

    # get some metadata
    spacing = img_sitk.GetSpacing()
    # the spacing returned by SimpleITK is in inverse order relative to the numpy array we receive. If we wanted to
    # resample the data and if the spacing was not isotropic (in BraTS all cases have already been resampled to 1x1x1mm
    # by the organizers) then we need to pay attention here. Therefore we bring the spacing into the correct order so
    # that spacing[0] actually corresponds to the spacing of the first axis of the numpy array
    spacing = np.array(spacing)[::-1]
    direction = img_sitk.GetDirection()
    origin = img_sitk.GetOrigin()

    original_shape = img_npy.shape
    
    metadata = {
    'spacing': spacing,
    'direction': direction,
    'origin': origin,
    'original_shape': original_shape
    }
    
    return img_sitk, img_npy, metadata  


img_sitk, img_npy, metadata = load_img(dir_img)

Pixel Type    2
Size          (288, 288, 135)
Origin        (134.70086669921875, -172.93592834472656, -103.38882446289062)
Spacing       (0.9722222089767456, 0.9722222089767456, 1.0499999523162842)
Direction     (-0.0, 0.0, -1.0, 1.0, -0.0, 0.0, 0.0, 1.0, -0.0)


In [6]:
def resample(image, transform):
    # Output image Origin, Spacing, Size, Direction are taken from the reference
    # image in this call to Resample
    reference_image = image
    interpolator = sitk.sitkCosineWindowedSinc
    default_value = 100.0
    return sitk.Resample(image, reference_image, transform,
                         interpolator, default_value)    

In [7]:
# Rotation working!. Check how to avoid the labels to be distorted!!


def affine_rotate(transform, degrees=45.0):
    new_transform = sitk.AffineTransform(transform)
    print(np.array(transform.GetMatrix()))
    matrix = np.array(transform.GetMatrix()).reshape((3,3))
    radians = -np.pi * degrees / 180.
    rotation = np.array([[np.cos(radians), -np.sin(radians), 0], 
                         [np.sin(radians), np.cos(radians), 0], 
                         [0, 0, 1]])  # https://stackoverflow.com/questions/9187387/3d-rotation-on-image
    new_matrix = np.dot(rotation, matrix)
    new_transform.SetMatrix(new_matrix.ravel())
    resampled = resample(img_sitk, new_transform)
    return resampled
    

affine_ro = sitk.AffineTransform(3)
rotated = affine_rotate(affine_ro)
sitk.WriteImage(rotated, 'affine_rotation_' + list_of_files[19].split('/')[11:][0][0:-7] + '.nii.gz', True) 

[1. 0. 0. 0. 1. 0. 0. 0. 1.]


In [7]:
def affine_scale(transform, x_scale=0.8, y_scale=0.7):
    new_transform = sitk.AffineTransform(transform)
    matrix = np.array(transform.GetMatrix()).reshape((3,3))
    matrix[0,0] = x_scale
    matrix[1,1] = y_scale
    new_transform.SetMatrix(matrix.ravel())
    resampled = resample(img_sitk, new_transform)
    return resampled 

affine_sca = sitk.AffineTransform(3)
scaled = affine_scale(affine_sca)
sitk.WriteImage(scaled, 'affine_scaling_' + list_of_files[19].split('/')[11:][0][0:-7] + '.nii.gz', True) 

(9,)


In [ ]:
# So, the mechanics for image tranformations is to first create the affineTransform, 
# set the matrix to the tranformation and then resample!

In [13]:
dimension = 3

# Physical image size corresponds to the largest physical size in the training set, or any other arbitrary size.
reference_physical_size = np.zeros(dimension)

# Create the reference image with a zero origin, identity direction cosine matrix and dimension     
reference_origin = np.zeros(dimension)
reference_direction = np.identity(dimension).flatten()

# Select arbitrary number of pixels per dimension, smallest size that yields desired results 
# or the required size of a pretrained network (e.g. VGG-16 224x224), transfer learning. This will 
# often result in non-isotropic pixel spacing.
reference_size = [128]*dimension 
reference_spacing = [ 1, 1, 1 ]

# Another possibility is that you want isotropic pixels, then you can specify the image size for one of
# the axes and the others are determined by this choice. Below we choose to set the x axis to 128 and the
# spacing set accordingly. 
# Uncomment the following lines to use this strategy.
#reference_size_x = 128
#reference_spacing = [reference_physical_size[0]/(reference_size_x-1)]*dimension
#reference_size = [int(phys_sz/(spc) + 1) for phys_sz,spc in zip(reference_physical_size, reference_spacing)]

reference_image = sitk.Image(reference_size, 2)
reference_image.SetOrigin(reference_origin)
reference_image.SetSpacing(reference_spacing)
reference_image.SetDirection(reference_direction)

# Always use the TransformContinuousIndexToPhysicalPoint to compute an indexed point's physical coordinates as 
# this takes into account size, spacing and direction cosines. For the vast majority of images the direction 
# cosines are the identity matrix, but when this isn't the case simply multiplying the central index by the 
# spacing will not yield the correct coordinates resulting in a long debugging session. 
reference_center = np.array(reference_image.TransformContinuousIndexToPhysicalPoint(np.array(reference_image.GetSize())/2.0))

In [14]:
def augment_images_spatial(original_image, reference_image, T0, T_aug, transformation_parameters,
                    output_prefix, output_suffix,
                    interpolator = sitk.sitkLinear, default_intensity_value = 0.0):
    '''
    Generate the resampled images based on the given transformations.
    Args:
        original_image (SimpleITK image): The image which we will resample and transform.
        reference_image (SimpleITK image): The image onto which we will resample.
        T0 (SimpleITK transform): Transformation which maps points from the reference image coordinate system 
            to the original_image coordinate system.
        T_aug (SimpleITK transform): Map points from the reference_image coordinate system back onto itself using the
               given transformation_parameters. The reason we use this transformation as a parameter
               is to allow the user to set its center of rotation to something other than zero.
        transformation_parameters (List of lists): parameter values which we use T_aug.SetParameters().
        output_prefix (string): output file name prefix (file name: output_prefix_p1_p2_..pn_.output_suffix).
        output_suffix (string): output file name suffix (file name: output_prefix_p1_p2_..pn_.output_suffix).
        interpolator: One of the SimpleITK interpolators.
        default_intensity_value: The value to return if a point is mapped outside the original_image domain.
    '''
    all_images = [] # Used only for display purposes in this notebook.
    for current_parameters in transformation_parameters:
        T_aug.SetParameters(current_parameters)        
        # Augmentation is done in the reference image space, so we first map the points from the reference image space
        # back onto itself T_aug (e.g. rotate the reference image) and then we map to the original image space T0.
        T_all = sitk.Transform(T0)
        T_all.AddTransform(T_aug)
        aug_image = sitk.Resample(original_image, reference_image, T_all,
                                  interpolator, default_intensity_value)
        sitk.WriteImage(aug_image, output_prefix + '_' + 
                        '_'.join(str(param) for param in current_parameters) +'_.' + output_suffix)
         
        all_images.append(aug_image) # Used only for display purposes in this notebook.
    return all_images # Used only for display purposes in this notebook.



In [22]:
from utilities import parameter_space_regular_grid_sampling, \
                      similarity3D_parameter_space_regular_sampling, eul2quat 


transformation_parameters_list = similarity3D_parameter_space_regular_sampling(np.linspace(-np.pi/18.0,np.pi/18.0,3),
                                                                               np.linspace(-np.pi/18.0,np.pi/18.0,3),
                                                                               np.linspace(-np.pi/18.0,np.pi/18.0,3),
                                                                               np.linspace(-10,10,3),
                                                                               np.linspace(-10,10,3),
                                                                               np.linspace(-10,10,3),
                                                                               np.linspace(0.9,1.1,3))
aug_transform = sitk.Similarity3DTransform()

# Transform which maps from the reference_image to the current img with the translation mapping the image
# origins to each other.
transform = sitk.AffineTransform(dimension)
transform.SetMatrix(img_sitk.GetDirection())
transform.SetTranslation(np.array(img_sitk.GetOrigin()) - reference_origin)
# Modify the transformation to align the centers of the original and reference image instead of their origins.
centering_transform = sitk.TranslationTransform(dimension)
img_center = np.array(img_sitk.TransformContinuousIndexToPhysicalPoint(np.array(img_sitk.GetSize())/2.0))
centering_transform.SetOffset(np.array(transform.GetInverse().TransformPoint(img_center) - reference_center))
centered_transform = sitk.Transform(transform)
centered_transform.AddTransform(centering_transform)

# Set the augmenting transform's center so that rotation is around the image center.
aug_transform.SetCenter(reference_center)

generated_image = augment_images_spatial(img_sitk, reference_image, centered_transform, 
                                       aug_transform, transformation_parameters_list, 
                                       os.path.join('./', 'spatial_aug_'), 'mha')

KeyboardInterrupt: 

In [ ]:
# Use sitk.sitkNearestNeighbor for labels!